In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def open_data(fname):
    df = pd.read_csv(fname, delimiter='\t')
    return df.values[:, :-1], df.values[:, -1]

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def plot_data(frontier, fname, method, X, y):
    fig, ax = plt.subplots(figsize=(7,7))
    fig.tight_layout()
    ax.scatter(X[:, 0], X[:, 1], c=y)
    ax.plot(np.linspace(X[:, 0].min(), X[:, 0].max()), 
            frontier(np.linspace(X[:, 0].min(), X[:, 0].max())), 'k')
    wi = X[:, 0].max() - X[:, 0].min()
    he = X[:, 1].max() - X[:, 1].min()
    ax.set_xlim(X[:, 0].min()-0.1*wi, X[:, 0].max()+0.1*wi)
    ax.set_ylim(X[:, 1].min()-0.1*he, X[:, 1].max()+0.1*he)
    plt.title(method)
    plt.savefig('{}_{}.pdf'.format(fname, method))
    plt.close()
    
def plot_mesh_data(frontier, fname, method, X, y):
    fig, ax = plt.subplots(figsize=(7,7))
    fig.tight_layout()
    ax.scatter(X[:, 0], X[:, 1], c=y)
    
    wi = X[:, 0].max() - X[:, 0].min()
    he = X[:, 1].max() - X[:, 1].min()
    
    x = np.linspace(X[:, 0].min()-0.1*wi, X[:, 0].max()+0.1*wi)
    y = np.linspace(X[:, 1].min()-0.1*he, X[:, 1].max()+0.1*he)
    x, y = np.meshgrid(x, y)
    
    ax.contour(x, y, frontier(x, y), [0.5], colors='k')

    ax.set_xlim(X[:, 0].min()-0.1*wi, X[:, 0].max()+0.1*wi)
    ax.set_ylim(X[:, 1].min()-0.1*he, X[:, 1].max()+0.1*he)
    plt.title(method)
    plt.savefig('{}_{}.pdf'.format(fname, method))
    
    plt.close()

In [4]:
class LDA_model:
    
    def fit(self, X, y):
        self.pi = np.sum(y==1)/len(y)
        self.mu1 = np.mean(X[y==1], axis=0)
        self.mu2 = np.mean(X[y==0], axis=0)
        
        sigma = np.cov(X[y==1].T)*np.sum(y==1)/len(y) + np.cov(X[y==0].T)*np.sum(y==0)/len(y)
        self.sigma_inv = np.linalg.inv(sigma)
        
        self.a = np.log(self.pi/(1-self.pi)) + (1/2)*(self.mu2.T.dot(self.sigma_inv).dot(self.mu2) - 
                                                      self.mu1.T.dot(self.sigma_inv).dot(self.mu1))
        self.b = (self.mu1 - self.mu2).T.dot(self.sigma_inv)
        
    def compute_frontier(self): 

        def frontier(x1):
            return (-self.b[0]/self.b[1]) * x1 + self.a/self.b[1]
        
        return frontier
        
    
    def predict_proba(self, X):
        return sigmoid(self.a + self.b.dot(X.T))
    
    def predict(self, X):
        return self.predict_proba(X) > 0.5
        
    def compute_misclassif_error(self, X, y):
        return np.sum(self.predict(X) != y)/len(y)

In [5]:
class QDA_model:
    
    def fit(self, X, y):
        self.pi = np.sum(y==1)/len(y)
        self.mu1 = np.mean(X[y==1], axis=0)
        self.mu2 = np.mean(X[y==0], axis=0)
        
        sigma1 = np.cov(X[y==1].T)
        sigma2 = np.cov(X[y==0].T)
        self.sigma1_inv = np.linalg.inv(sigma1)
        self.sigma2_inv = np.linalg.inv(sigma2)
        
        self.a = np.log(self.pi/(1-self.pi)) + (1/2)*(self.mu2.T.dot(self.sigma2_inv).dot(self.mu2) - 
                                            self.mu1.T.dot(self.sigma1_inv).dot(self.mu1))
        self.b = self.mu1.T.dot(self.sigma1_inv) - self.mu2.T.dot(self.sigma2_inv)
        
    def compute_frontier(self): 

        def frontier(x1, x2):
            interm = np.concatenate((x1[:, :, np.newaxis], x2[:, :, np.newaxis]), axis=2).reshape(-1, 2)
            return self.predict_proba(interm).reshape(x1.shape)
        
        return frontier
    
    def predict_proba(self, X):
        return sigmoid(self.a + self.b.dot(X.T) + 
                       (1/2)*np.sum(X.dot(self.sigma2_inv-self.sigma1_inv)*X, axis=1))
    
    def predict(self, X):
        return self.predict_proba(X) > 0.5
        
    def compute_misclassif_error(self, X, y):
        return np.sum(self.predict(X) != y)/len(y)

In [15]:
class Logistic:
    def __init__(self, reg=0):
        self.reg=reg
    
    def fit(self, X, y, tol=1e-4):
        X = np.concatenate((np.ones_like(X[:, 0])[:, np.newaxis], X), axis=1)
        start=True
        w = np.zeros(X[0, :].T.shape)
        itern = 0
        if start:
            w_new = w - 2*tol
        
        while np.linalg.norm(w_new - w) > tol:# and itern < 1000:
            w = w_new.copy()
            eta = sigmoid(w.dot(X.T))
            gradl = X.T.dot(y-eta)
            Hl = -X.T.dot(np.diag(eta*(1-eta))).dot(X)
            w_new = w - 0.01*np.linalg.inv(Hl + self.reg*np.identity(Hl.shape[0])).dot(gradl) #
            
            itern += 1
            if itern%1000 == 0: print(w)
            #print(w, eta, gradl, Hl)
            #break
        self.w = w_new
        
    def compute_frontier(self): 

        def frontier(x1):
            return (1/self.w[2])*(-self.w[1] * x1 - self.w[0])
        return frontier
        
    def predict_proba(self, X):
        X = np.concatenate((np.ones_like(X[:, 0])[:, np.newaxis], X), axis=1)
        return sigmoid(self.w.dot(X.T))
    
    def predict(self, X):
        return self.predict_proba(X) > 0.5
        
    def compute_misclassif_error(self, X, y):
        return np.sum(self.predict(X) != y)/len(y)

In [16]:
class LinearRegression:
    def fit(self, X, y):
        X = np.concatenate((np.ones_like(X[:, 0])[:, np.newaxis], X), axis=1)
        self.w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        
    def compute_frontier(self): 

        def frontier(x1):
            return (1/self.w[2])*(-self.w[1] * x1 + 0.5 - self.w[0])
        
        return frontier
    
    def predict_proba(self, X):
        X = np.concatenate((np.ones_like(X[:, 0])[:, np.newaxis], X), axis=1)
        return self.w.dot(X.T)
    
    def predict(self, X):
        return self.predict_proba(X) > 0.5
        
    def compute_misclassif_error(self, X, y):
        return np.sum(self.predict(X) != y)/len(y)

In [17]:
reg

NameError: name 'reg' is not defined

In [19]:
for name in ['classificationA', 'classificationB', 'classificationC']:
    X, y = open_data('classification_data_HWK1/{}.train'.format(name))
    X_test, y_test = open_data('classification_data_HWK1/{}.test'.format(name))
    print(name)
    
    ## LDA
    lda = LDA_model()
    lda.fit(X, y)
    plot_data(lda.compute_frontier(), name, 'lda', X, y)
    
    print("LDA Train Error: {:.10f}".format(lda.compute_misclassif_error(X, y)))
    print("LDA Test Error: {:.10f}\n".format(lda.compute_misclassif_error(X_test, y_test)))
    
    ## Logistic Regression
    logistic = Logistic(reg=0)
    logistic.fit(X, y)
    plot_data(logistic.compute_frontier(), name, 'logistic', X, y)
    
    print("Logistic Regression Train Error: {:.10f}".format(logistic.compute_misclassif_error(X, y)))
    print("Logistic Regression Test Error: {:.10f}\n".format(logistic.compute_misclassif_error(X_test, y_test)))
    
    ## Linear Regression
    linear = LinearRegression()
    linear.fit(X, y)
    plot_data(linear.compute_frontier(), name, 'linear', X, y)
    
    print("Linear Regression Train Error: {:.10f}".format(linear.compute_misclassif_error(X, y)))
    print("Linear Regression Test Error: {:.10f}\n".format(linear.compute_misclassif_error(X_test, y_test)))
    
    ## QDA Regression
    qda = QDA_model()
    qda.fit(X, y)
    plot_mesh_data(qda.compute_frontier(), name, 'qda', X, y)
    
    print("QDA Train Error: {:.4f}".format(qda.compute_misclassif_error(X, y)))
    print("QDA Test Error: {:.4f}\n".format(qda.compute_misclassif_error(X_test, y_test)))
    
    # Generate the error tables
    with open(name+'_table.txt', 'w') as f:
        f.write("""
    \\begin{{tabular}}{{c|cccc}}
    Method & LDA & Logistic & Linear & QDA \\\\
    \hline
    Train Error & {:.4f} & {:.4f} & {:.4f} & {:.4f} \\\\
    \hline
    Test Error & {:.4f} & {:.4f} & {:.4f} & {:.4f} \\\\
    \\end{{tabular}}""".format(
        lda.compute_misclassif_error(X, y),
        logistic.compute_misclassif_error(X, y),
        linear.compute_misclassif_error(X, y),
        qda.compute_misclassif_error(X, y),
        lda.compute_misclassif_error(X_test, y_test),
        logistic.compute_misclassif_error(X_test, y_test),
        linear.compute_misclassif_error(X_test, y_test),
        qda.compute_misclassif_error(X_test, y_test)))
    
    print()

classificationA
LDA Train Error: 0.0134228188
LDA Test Error: 0.0206804536

[ -13.85898798  -80.85535669 -139.55936297]
[ -59.23576458 -355.95740408 -616.43986206]
[ -104.53911804  -630.51749297 -1092.32756961]
[ -149.84248379  -905.07765223 -1568.21539316]
[ -191.61639924 -1137.15534938 -1976.17879187]
Logistic Regression Train Error: 0.0000000000
Logistic Regression Test Error: 0.0353569046

Linear Regression Train Error: 0.0134228188
Linear Regression Test Error: 0.0206804536

QDA Train Error: 0.0067
QDA Test Error: 0.0193


classificationB
LDA Train Error: 0.0301003344
LDA Test Error: 0.0415207604

Logistic Regression Train Error: 0.0200668896
Logistic Regression Test Error: 0.0435217609

Linear Regression Train Error: 0.0301003344
Linear Regression Test Error: 0.0415207604

QDA Train Error: 0.0234
QDA Test Error: 0.0235


classificationC
LDA Train Error: 0.0551378446
LDA Test Error: 0.0420140047

Logistic Regression Train Error: 0.0401002506
Logistic Regression Test Error: 0.02267